# Chunking and SQLite

In this notebook, we'll see a couple of techniques that can be used whe working with large file in Python.

Specifically, we'll be looking at the dataset of Metro Nashville Police Department Calls for Service (https://data.nashville.gov/Police/Metro-Nashville-Police-Department-Calls-for-Servic/kwnd-qrrm) with the goal being to look at specific types of calls for service and to bring in additional information, when possible, from the Metro Nashville Police Department Incidents file (https://data.nashville.gov/Police/Metro-Nashville-Police-Department-Incidents/2u6v-ujjs).

In [ ]:
import pandas as pd

Before we do anything else, let's see how many rows are contained in the Calls for Service file. 

We can start by reading in just a few rows to get an idea of what the dataset we are working with looks like. This can be done using the `nrows` argument.

In [ ]:
metro_calls = pd.read_csv('data/Metro_Nashville_Police_Department_Calls_for_Service.csv', 
                          nrows = 10000)

In [ ]:
metro_calls.head()

Let's say that our goal is to find all incidents where the Tencode Description was 'SHOTS FIRED'.

One thing we could try is to use the chunksize argument in our pd.read_csv call. What this does is to create an iterable which returns just the specified number of rows at a time.

Iterating through a file using chunks can look like this, but can also be structured differently (for example, using a list comprehension).

``` 
chunks = pd.read_csv('data/Metro_Nashville_Police_Department_Calls_for_Service.csv', chunksize = 10000)

for chunk in chunks:
    # Do something 
```

Here is what we need to do:

1. Create an iterable by using the chunksize argument.

2. For each chunk, filter to just the rows where the 'Tencode Description' column is 'SHOTS FIRED'. Store these rows.

3. Concatenate all the results together into a single dataframe.

In [ ]:
# Your code here.

The above solution would work for one-off tasks. However, if you are going to be working extensively with a dataset or merging two large datasets, it would be a bit cumbersome to have to chunk through one or both datasets multiple times.

As an alternative to working in just Python, we can use a different tool which works better on large datasets - SQL.

In this notebook, we will make use of SQLite, which is a file-based relational database management system. We can interact with SQLite databases through the sqlite3 library.

In [ ]:
import sqlite3

First, we need to connect to our database. The connect function will either create a new database if one does not already exist or connect to an existing one.

In [ ]:
db = sqlite3.connect('data/police_calls.sqlite')

Now, we can chunk through the data and for each row, add the rows to a table in our sqlite database. 
To keep track of how much progress has been made, we can use the `tqdm` library.

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
for chunk in tqdm(pd.read_csv('data/Metro_Nashville_Police_Department_Calls_for_Service.csv', chunksize = 10000)):
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]      # Clean up the column names
    chunk.to_sql('calls', db, if_exists = 'append', index = False)            # Append the chunk to a calls table

To speed up queries which use a specific column, we can create an **index** on that column. This causes the database to store that column in a way that helps it to retrieve rows quicker.

In [ ]:
db.execute('CREATE INDEX tencode_description ON calls(tencode_description)')

Finally, we should close our database connection.

In [ ]:
db.close()

Now, let's see how long it takes to find all rows corresponding to 'SHOTS FIRED'.

In [ ]:
db = sqlite3.connect('data/police_calls.sqlite')

query = "SELECT * FROM calls WHERE tencode_description = 'SHOTS FIRED'"

shots_sqlite = pd.read_sql(query, db)

db.close()

In [ ]:
shots_sqlite.head()

The Metro Police Department Incidents database 

In [ ]:
# Your code here

If we want to match calls to incidents, we can use the complaint_number column from the calls database and the incident_number column from the incidents database.

To speed up this process, we can created indexes on these two columns.

In [ ]:
db = sqlite3.connect('data/police_calls.sqlite')

db.execute('CREATE INDEX complaint_number ON calls(complaint_number)')
db.execute('CREATE INDEX incident_number ON incidents(incident_number)')

db.close()

Now, let's grab all SHOTS FIRED calls for which there is an associated incident and bring in the incident information.

In [ ]:
# Your code here.